In [2]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from torch.utils.data import random_split, DataLoader, TensorDataset
%matplotlib inline

import torch.optim as optim
import time
# import torchvision.models as models

In [3]:
train_ds= torchvision.datasets.CIFAR10(root="data/", train = True, transform=transforms.ToTensor(), download=True)
test_ds= torchvision.datasets.CIFAR10(root="data/", train = False, transform=transforms.ToTensor(), download=True)

Files already downloaded and verified
Files already downloaded and verified


In [1]:
# len(train_ds) = 50000
# len(train_loader) = 391

In [4]:
batch_size=128
train_loader= DataLoader(train_ds, batch_size, shuffle =True)
test_leader=  DataLoader(test_ds, batch_size, shuffle = False)
# len(train_loader)

In [5]:
device=torch.device("cuda:1" if torch.cuda.is_available else "cpu")

def imshow(img):
    img = img / 2 + 0.5  # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


num_classes=10
classes = ('plane', 'car', 'bird', 'cat','deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [6]:
class Bottleneck(nn.Module):
    def __init__(self, inplane, planes, stride=1):
        super(Bottleneck,self).__init__()
        outplane=4*planes
        self.conv1 =   nn.Conv2d(inplane, planes,kernel_size=1,bias=False)  #same convolution
        self.batchN1 = nn.BatchNorm2d(planes)
        self.conv2 =   nn.Conv2d(planes, planes,kernel_size=3 ,stride=stride, padding=1 ,bias=False) #if stride is 1 then same convolution
        self.batchN2 =  nn.BatchNorm2d(planes)
        self.conv3 =   nn.Conv2d(planes, outplane,kernel_size=1,bias=False) #same convolution
        self.batchN3 =  nn.BatchNorm2d(outplane)
        
        self.shortcuts=nn.Sequential()  #creating a shortcut which will help in residual part
        if(outplane!=inplane or stride!=1):  #we make the shortcut so that the dimension match 
            self.shortcuts = nn.Sequential(nn.Conv2d(inplane, outplane, kernel_size=1,stride=stride,bias=False) ,nn.BatchNorm2d(outplane))
            
    def forward(self, x):
        out=F.relu(self.batchN1(self.conv1(x)))
        out=F.relu(self.batchN2(self.conv2(out)))
        out=self.batchN3(self.conv3(out))
        out=F.relu(out+self.shortcuts(x))   #most important step of residual network
        return out


In [7]:
class Resmodel(nn.Module):
    # def __init__(self, block=Bottleneck, no_bl=[3,4,6,3], output_no=10):
    def __init__(self, block, no_bl=[3,4,6,3], output_no=10):
        super(Resmodel,self).__init__()
        self.in_plane=64
        self.conv1= nn.Conv2d(3,64,kernel_size=3,stride=1,padding=1 ,bias=False)
        self.batchN11= nn.BatchNorm2d(64)
        self.layer1= self._make_layer(block, 64, 3, 1)
        self.layer2= self._make_layer(block, 128, 4, 2)
        self.layer3= self._make_layer(block, 256, 6, 2)
        self.layer4= self._make_layer(block, 512, 3, 2)
        self.linear = nn.Linear(2048,10)    #finally we will classify in 10 class,hence we have 10 dimension in final layer
        
    def _make_layer(self, block, plane, numofblock, stride):
        layers=[]
        strides= [stride] + [1]*(numofblock-1) #first block will have the given stride rest will have stride=1
        #we have stride only for the first bottleneck(that too only one layer of bottleneck will use it)(hence the reduction of dimension due to stride will take place only once for one block of bottleneck)
        for st in strides:
            layers.append(Bottleneck(self.in_plane, plane, st))
            self.in_plane = plane*4     #we know that the last layer in bottleneck is 4 times the size of input layer, hence we multiply it here, so that the first layer of next bottleneck will match the dimension of last layer of previous bottleneck.
        return nn.Sequential(*layers)   #*layers will return values stored in the list
    
    def forward(self, x):
        #size of x=[batchsize,3,32,32]
        out=self.conv1(x)           #after this operation size=[batchsize,64,32,32]
        out=self.batchN11(out)
        out=F.relu(out)
        # out= F.relu(self.batchN11(self.conv1(x)))
        out=self.layer1(out)        #after this operation the output size must be [batchsize,256,32,32]
        out=self.layer2(out)        #after this operation the output size must be [batchsize,512,16,16]
        out=self.layer3(out)        #after this operation the output size must be [batchsize,1024,8,8]
        out=self.layer4(out)        #after this operation the output size must be [batchsize,2048,4,4]
        out=F.avg_pool2d(out,4)     #after this operation the output size must be [batchsize,2048,1,1]
        out= out.view(out.size(0),-1)    #after this operation the output size must be [batchsize,1,2048]
        out=self.linear(out)         #after this operation the output size must be [batchsize,1,10]
        return out


In [8]:
model = Resmodel(Bottleneck)
model=model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer= optim.SGD(model.parameters(), lr= 0.01)


In [23]:
%%time
n_total_steps = len(train_loader)
for epoch in range(2):
    for i, (images, labels) in enumerate(train_loader):
        # origin shape: [128, 3, 32, 32]
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if (i+1) % 16 == 0:
            print (f'Epoch [{epoch+1}/{2}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

# print('max value of i=',max_val_i)
print('Finished Training')
PATH = './cnn.pth'
torch.save(model.state_dict(), PATH)




Epoch [1/2], Step [16/391], Loss: 0.0922
Epoch [1/2], Step [32/391], Loss: 0.1403
Epoch [1/2], Step [48/391], Loss: 0.0884
Epoch [1/2], Step [64/391], Loss: 0.1015
Epoch [1/2], Step [80/391], Loss: 0.1626
Epoch [1/2], Step [96/391], Loss: 0.1563
Epoch [1/2], Step [112/391], Loss: 0.1998
Epoch [1/2], Step [128/391], Loss: 0.1411
Epoch [1/2], Step [144/391], Loss: 0.1463
Epoch [1/2], Step [160/391], Loss: 0.2057
Epoch [1/2], Step [176/391], Loss: 0.1315
Epoch [1/2], Step [192/391], Loss: 0.1081
Epoch [1/2], Step [208/391], Loss: 0.1253
Epoch [1/2], Step [224/391], Loss: 0.1002
Epoch [1/2], Step [240/391], Loss: 0.2070
Epoch [1/2], Step [256/391], Loss: 0.1577
Epoch [1/2], Step [272/391], Loss: 0.1727
Epoch [1/2], Step [288/391], Loss: 0.3206
Epoch [1/2], Step [304/391], Loss: 0.1674
Epoch [1/2], Step [320/391], Loss: 0.1892
Epoch [1/2], Step [336/391], Loss: 0.2041
Epoch [1/2], Step [352/391], Loss: 0.0818
Epoch [1/2], Step [368/391], Loss: 0.1765
Epoch [1/2], Step [384/391], Loss: 0.295

In [24]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images, labels in test_leader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)

        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        
        # for i in range(batch_size):
        for i in range(len(labels)):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')

    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}: {acc} %')


Accuracy of the network: 69.88 %
Accuracy of plane: 71.2 %
Accuracy of car: 85.5 %
Accuracy of bird: 65.5 %
Accuracy of cat: 69.3 %
Accuracy of deer: 65.9 %
Accuracy of dog: 27.6 %
Accuracy of frog: 79.3 %
Accuracy of horse: 69.4 %
Accuracy of ship: 85.2 %
Accuracy of truck: 79.9 %


In [19]:
# len(test_leader) = 79
# len(images)
# 79*128 almost equals 10k
# print(images.shape)

In [20]:


# y = torch.randn(1, 3, 4,4)
# out=F.avg_pool2d(y,4)
# print(out.shape)
# out= out.view(out.size(0),-1)
# print("new shape=",out.shape)
